### Assignment 6
    Akhil Kanna Devarashetti

Question:

    Write a Pytorch version of the Word2vec/skip-gram displayed in Chapter 14 of d2l.  
    In particular, make DL24.py error free 
    Implement get_similar_tokens as an application of the word embedding model 
    (14.4.3 of d2l and also the last slide in lecture)


In [1]:
import collections
import random
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

In [2]:
raw_text = open('ptb/ptb.train.txt', "r").read()
#raw_text = raw_text[:600]

In [3]:
## Get the sentences and tokens (words)
sentences = [line.split() for line in raw_text.split('\n')]
tokens = [tk for line in sentences for tk in line]
num_tokens = len(tokens)

# Get unique tokens (words) with count > 10
counter = collections.Counter(tokens)
uniq_tokens = [token for token, freq in list(counter.items()) if counter[token] >= 0]  # Make counter 10

# Create hash map of the unique words and indices
idx_to_token, token_to_idx = uniq_tokens, dict()
for i in range(len(uniq_tokens)):
    token_to_idx[uniq_tokens[i]] = i

In [4]:
subsampled = []

for line in sentences:
    sub_sampled_line = []
    for token in line:
        random_number = random.uniform(0, 1)

        order_of_magnitude = round(math.log10(num_tokens))
        inverse_frequency = num_tokens / (10 ** order_of_magnitude) / counter[token]
        
        if random_number < inverse_frequency:
            sub_sampled_line.append(token)
    subsampled.append(sub_sampled_line)


In [5]:
corpus = [[token_to_idx.get(tk) for tk in line] for line in subsampled]
tokens = [tk for line in corpus for tk in line]
counter = collections.Counter(tokens)
sampling_weights = [counter[i]**0.75 for i in range(len(counter))]
population = list(range(len(sampling_weights)))
candidates = random.choices(population, sampling_weights, k=(10**order_of_magnitude))

In [6]:
max_window_size = 5
K = 5
j = 0
data = []
maxLen = 0
for line in corpus:
    if len(line) < 2:
        continue
    for i in range(len(line)):
        window_size = random.randint(1, max_window_size)
        indices = list(range(max(0, i - window_size), min(len(line), i + 1 + window_size)))
        indices.remove(i)
        for idx in indices:
            context = [line[idx] for idx in indices]
        neg = []
        while len(neg) < len(context) * K:
            ne = candidates[j]
            j += 1
            if j >= 10**order_of_magnitude:
                j = 0
            if ne not in context:
                neg.append(ne)
        data.append([line[i], context, neg])

In [7]:
max_len = max(len(c) + len(n) for _, c, n in data)
centers, contexts_negatives, labels = [], [], []
for center, context, negative in data:
    cur_len = len(context) + len(negative)
    centers += [center]
    contexts_negatives += [context + negative + [0] * (max_len - cur_len)]
    labels += [[1] * len(context) + [0] * (max_len - len(context))]

In [8]:
class PTBdataset(torch.utils.data.Dataset):
    def __init__(self):
        super(PTBdataset).__init__()
        self.centers = np.array(centers).reshape(-1, 1)
        self.contexts_negatives = np.array(contexts_negatives)
        self.labels = np.array(labels)

    def __len__(self):
        return len(self.centers)

    def __getitem__(self, idx):
        return self.centers[idx], self.contexts_negatives[idx], self.labels[idx]

In [9]:
pdata = PTBdataset()
data_iter = torch.utils.data.DataLoader(pdata, batch_size=512, shuffle=True)

vocab_size = len(idx_to_token)
embed_size = 100

In [10]:
net = nn.Sequential(
    nn.Embedding(vocab_size, embed_size),
    nn.Embedding(vocab_size, embed_size))
loss = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), 0.01)
m = nn.Sigmoid()

In [11]:
for epoch in range(500):
    for i, batch in enumerate(data_iter):
        center, context_negative, label = batch
        v = net[0](center.to(torch.int64))
        u = net[1](context_negative.to(torch.int64))
        pred = torch.tensordot(v, torch.transpose(u, 1, 2))
        l = loss(m(pred), label.to(torch.float32))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        if (epoch + 1) % 50 == 0:
            print(epoch, i, float(l))

49 0 13.973994255065918
49 1 13.450678825378418
49 2 13.671319007873535
49 3 13.562156677246094
99 0 13.296063423156738
99 1 13.219744682312012
99 2 13.891067504882812
99 3 13.918510437011719
149 0 13.91940689086914
149 1 12.823521614074707
149 2 13.389513969421387
149 3 14.116114616394043
199 0 13.289380073547363
199 1 14.042492866516113
199 2 13.431744575500488
199 3 13.153682708740234
249 0 13.435328483581543
249 1 13.22641658782959
249 2 14.033463478088379
249 3 13.312944412231445
299 0 12.956456184387207
299 1 13.978198051452637
299 2 12.850151062011719
299 3 14.403619766235352
349 0 13.297236442565918
349 1 13.278010368347168
349 2 14.101157188415527
349 3 13.244428634643555
399 0 13.440058708190918
399 1 13.90501880645752
399 2 13.10141658782959
399 3 13.590893745422363
449 0 13.274828910827637
449 1 14.13455581665039
449 2 13.366667747497559
449 3 13.13536262512207
499 0 13.166953086853027
499 1 13.699658393859863
499 2 13.853084564208984
499 3 13.196040153503418


In [33]:
# Derived from the code in the link: https://d2l.ai/chapter_natural-language-processing-pretraining/similarity-analogy.html#finding-synonyms

def knn(W, x, k):
    cos = torch.matmul(W, x.reshape(-1,)) / (
        (torch.sum(W * W, dim=1) + 1e-9).sqrt() * torch.sum(x * x).sqrt())
    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    return topk, [cos[i].item() for i in topk]

def get_similar_tokens(query_token, k, embed):
    W = embed.weight.data
    x = W[token_to_idx[query_token]]
    topk, cos = knn(W, x, k+1)
    for i, c in zip(topk[1:], cos[1:]):
        print('cosine sim=%.3f: %s' % (c, (idx_to_token[i])))

get_similar_tokens('group', 3, net[0])

cosine sim=0.482: syndrome
cosine sim=0.395: budgetary
cosine sim=0.330: vision


In [31]:
cos_function = torch.nn.CosineSimilarity(dim=1, eps=1e-08)

def knn(W, x, k):
    print(f"W shape = {W.shape}")
    print(f"x shape = {x.shape}")
    cos = cos_function(W, x.reshape(-1,))
    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    return topk, [cos[i].item() for i in topk]

In [ ]:
words_for_similarity = ['used', 'this', 'director', 'chairman', 'is', 'was']

for word in words_for_similarity:
    print(f"\nSimilarity for '{word}':")
    get_similar_tokens(word, 3, net[0])
